In [1]:
API_KEY="rfnqn5jyts2ljcq1"
ACCESS_TOKEN="RUBbPfsVE3sbiBpIk109eAY4L8cOvdnv"

headers={"X-Kite-Version":'3','Authorization':"token "+API_KEY+":"+ACCESS_TOKEN}
ROOT_ENDPOINT="https://api.kite.trade"

import requests as re
from pprint import pprint as pp

In [7]:
import datetime
import pandas as pd

In [20]:
def getData(token, fromm, to, time_frame):
    
    data=re.get(f"https://api.kite.trade/instruments/historical/{token}/{time_frame}?from={fromm}&to={to}",headers=headers)
    print("data-->", data.json())
    df=pd.DataFrame(data.json()['data']['candles'])
    df.columns = ["Timestamp", "open", "high", "low","close","volume"]

    return SuperTrend(df).iloc[-1]

In [21]:
def SuperTrend(data, period = 10, multiplier=3, ohlc=['open', 'high', 'low', 'close']):
        global supertrend
        data['tr0'] = abs(data["high"] - data["low"])
        data['tr1'] = abs(data["high"] - data["close"].shift(1))
        data['tr2'] = abs(data["low"]- data["close"].shift(1))
        data["TR"] = round(data[['tr0', 'tr1', 'tr2']].max(axis=1),2)
        data["ATR"]=0.00
        data['BUB']=0.00
        data["BLB"]=0.00
        data["final_ub"]=0.00
        data["final_lb"]=0.00
        data["ST"]=0.00
        for i, row in data.iterrows():
            if i == 0:
                data.loc[i,'ATR'] = 0.00#data['ATR'].iat[0]
            else:
                data.loc[i,'ATR'] = ((data.loc[i-1,'ATR'] * (period-1))+data.loc[i,'TR'])/period
        data['BUB'] = round(((data["high"] + data["low"]) / 2) + (multiplier * data["ATR"]),2)
        data['BLB'] = round(((data["high"] + data["low"]) / 2) - (multiplier * data["ATR"]),2)
        for i, row in data.iterrows():
            if i==0:
                data.loc[i,"final_ub"]=0.00
            else:
                if (data.loc[i,"BUB"]<data.loc[i-1,"final_ub"])|(data.loc[i-1,"close"]>data.loc[i-1,"final_ub"]):
                    data.loc[i,"final_ub"]=data.loc[i,"BUB"]
                else:
                    data.loc[i,"final_ub"]=data.loc[i-1,"final_ub"]

        for i, row in data.iterrows():
            if i==0:
                data.loc[i,"final_lb"]=0.00
            else:
                if (data.loc[i,"BLB"]>data.loc[i-1,"final_lb"])|(data.loc[i-1,"close"]<data.loc[i-1,"final_lb"]):
                    data.loc[i,"final_lb"]=data.loc[i,"BLB"]
                else:
                    data.loc[i,"final_lb"]=data.loc[i-1,"final_lb"]

        for i, row in data.iterrows():
            if i==0:
                data.loc[i,"ST"]=0.00
            elif (data.loc[i-1,"ST"]==data.loc[i-1,"final_ub"]) & (data.loc[i,"close"]<=data.loc[i,"final_ub"]):
                data.loc[i,"ST"]=data.loc[i,"final_ub"]
            elif (data.loc[i-1,"ST"]==data.loc[i-1,"final_ub"])&(data.loc[i,"close"]>data.loc[i,"final_ub"]):
                data.loc[i,"ST"]=data.loc[i,"final_lb"]
            elif (data.loc[i-1,"ST"]==data.loc[i-1,"final_lb"])&(data.loc[i,"close"]>=data.loc[i,"final_lb"]):
                data.loc[i,"ST"]=data.loc[i,"final_lb"]
            elif (data.loc[i-1,"ST"]==data.loc[i-1,"final_lb"])&(data.loc[i,"close"]<data.loc[i,"final_lb"]):
                data.loc[i,"ST"]=data.loc[i,"final_ub"]

        # Buy Sell Indicator
        for i, row in data.iterrows():
            if i==0:
                data["STX"]="NA"
            elif (data.loc[i,"ST"]<data.loc[i,"close"]) :
                data.loc[i,"STX"]="green"
            else:
                data.loc[i,"STX"]="red"
        data.drop(['tr0', 'tr1', 'tr2', 'TR','BUB', 'BLB'], inplace=True, axis=1)
        return data

In [22]:
def place_order(ROOT_ENDPOINT, ):
    data={
    "tradingsymbol":"ACC",
    "exchange":"NSE",
    "transaction_type":"SELL",
    "order_type":"LIMIT",
    "price":"1800",
    "quantity":"10000",
    "product":"MIS",
    "validity":"DAY"
    
}
    res=re.post(ROOT_ENDPOINT+'/orders/regular',headers=headers,data=data)
    print("order successful")
    return res
    

In [23]:
from dateutil import parser
import requests as re
API_KEY="rfnqn5jyts2ljcq1"
# RUBbPfsVE3sbiBpIk109eAY4L8cOvdnv
# VkjKYp51T2TgvvHNPukdY5K2gRttR3W7
ACCESS_TOKEN="e06AZ81g3j7VCITM3qcyrRoYjSNbhvA9"
headers={"X-Kite-Version":'3','Authorization':"token "+API_KEY+":"+ACCESS_TOKEN}
ROOT_ENDPOINT="https://api.kite.trade"
API_SECRET="4t8tlmijcewpoov8nayd978x7bpj4yif"



interval=3
import datetime,time

from_date = datetime.datetime.now()-datetime.timedelta(days=90)
to_date = datetime.datetime.now()-datetime.timedelta(minutes=interval)
if interval==1:
    time_frame = 'minute'
    
else:
    time_frame = str(interval)+'minute'
    
last_row = getData(5097729,from_date, to_date, time_frame )
last_time=last_row.Timestamp
last_signal=last_row.STX
print('running',datetime.datetime.now(),last_time,last_row.Timestamp,last_row.open,last_row.high,last_row.low,last_row.close,last_row.volume,last_row.ST,last_row.STX,'side=sell'if last_row.STX=='red' else 'side=buy','qty=1','change=True')
# place_order("NSE","ICICIBANK",1,'SELL')
# running 2023-03-24 13:58:41.163117 2023-03-24T13:54:00+0530 3357.65 3359.65 3355.3 3357.8 424 3367.48 red side=sell qty 1or2 change=True

while True:
    time.sleep(1)
    if str(parser.parse(last_time.replace("T"," "))+datetime.timedelta(minutes=interval*2))<str(datetime.datetime.now()):
        
        from_date = datetime.datetime.now()-datetime.timedelta(days=90)
        to_date = datetime.datetime.now()-datetime.timedelta(minutes=interval)
        last_row = getData(5097729,from_date, to_date, time_frame )
        last_time=last_row.Timestamp
        print('running',datetime.datetime.now(),last_time,last_row.Timestamp,last_row.open,last_row.high,last_row.low,last_row.close,last_row.volume,last_row.ST,last_row.STX,'side=sell'if last_row.STX=='red' else 'side=buy','qty=2','change=True' if last_row.STX!=last_signal else 'change=False')
        last_signal=last_row.STX
        if last_row.STX!=last_signal:
            print("oo yes")
#             order_plc =  place_order(ROOT_ENDPOINT,headers,"NSE","ICICIBANK",2,'BUY')
#             order_plc

        #         if stx="red":
#             print("side=sell")
#             else:
#                 print("side=buy")

data--> {'status': 'error', 'message': 'Incorrect `api_key` or `access_token`.', 'data': None, 'error_type': 'TokenException'}


TypeError: 'NoneType' object is not subscriptable